# ***MODOAP - Objects detection in images***

This notebook detects and extracts objects in a given images dataset located on a Google Drive. 

Detected objects correspond to the 80 classes of the COCO dataset : 

```
# class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']
```




The runtime environment must be set to GPU : (Runtime -> Change Runtime Type -> GPU)

In [ ]:
#@title ##  Connection to Google Drive

#@markdown ##### Execute the cell and connect to a Google account

import os
from google.colab import drive

if not os.path.exists("/content/drive/My Drive"):
  drive.mount('/content/drive')
else : print("Google Drive already mounted")

In [ ]:
#@title ##  Installation and dataset preparation

#@markdown ##### This cell installs and prepares the algorithm on a distant file system.

#@markdown ##### Enter the path to a dataset directory containing image files and execute the cell :

image_directory = "" #@param {type:"string"}

#@markdown ##### You can browse your Drive files in the left window "Files", then right click on a folder -> copy path

#@markdown ##### Path example : /content/drive/MyDrive/folder_containing_pictures

dossier_corpus = image_directory
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt


%cd
if not os.path.exists("/root/Mask_RCNN"):

  !git clone --quiet https://github.com/matterport/Mask_RCNN.git
  %cd /root/Mask_RCNN
  !pip install -q PyDrive
  #!pip install -r requirements.txt
  !pip install numpy
  !pip install scipy
  !pip install Pillow
  !pip install cython
  !pip install  matplotlib
  !pip install scikit-image
  !pip install tensorflow>=1.3.0

  !pip install keras>=2.0.8

  !pip install opencv-python

  !pip install h5py==2.10.0
  !pip install imgaug
  !pip install IPython[all]


  !python setup.py install
else : print("Algorithm already downloaded")

##########################################################

%cd /root/Mask_RCNN
###############################################################
%tensorflow_version 1.x
!pip install q keras==2.1.5
!pip install q keras==2.1.5
###############################################################


# Root directory of the project
ROOT_DIR = os.path.abspath("./")
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

%matplotlib inline

if not os.path.exists(os.path.join(dossier_corpus,"model")):
  os.makedirs(os.path.join(dossier_corpus,"model"))

# Directory to save logs and trained model
MODEL_DIR = os.path.join(dossier_corpus,"model")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = dossier_corpus

###############################################################

class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

###############################################################


# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

###############################################################

# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

In [ ]:
#@title ##  Objects Detection

#@markdown ##### This cell runs objects detection on the images of the dataset, and generates a .json file containing the annotations of the detected objects in the dataset folder/objects_annotations.


import glob
import cv2
import json
corpus = [image for image in glob.glob(dossier_corpus+"/*.*")]
dicocorpus = {}
print("number of images : ", len(corpus))
for nomfichier in corpus : 
  image = cv2.imread(nomfichier)
  results = model.detect([image], verbose=0)
  r = results[0]
  listeobjets = [vecteur for vecteur in r["rois"]]
  dicoimage={}

  visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                              class_names, r['scores'])

  for i in range(len(r["rois"])) :
    vecteur_region = [str(coord) for coord in listeobjets[i]]
    classe = class_names[r["class_ids"][i]]
    score = str(r["scores"][i])
    dicoobjet = {"regions":vecteur_region, "classe" : classe, "score":score}
    dicoimage["objet"+str(i)] = dicoobjet
  dicocorpus[nomfichier] = dicoimage

if not os.path.exists(os.path.join(dossier_corpus,"object_annotations")):
  os.makedirs(os.path.join(dossier_corpus,"object_annotations"))
json_annotations = os.path.join(dossier_corpus,"object_annotations", "object_annotations.json")
with open(json_annotations, "w") as dic :
  json.dump(dicocorpus, dic)

In [ ]:
#@title ##  Extraction of detected objects as images

#@markdown ##### This cell extracts the detected objects as new image files in the dataset directory/extracted_objects




import json
from google.colab.patches import cv2_imshow

with open(json_annotations, "r") as mondic :
  dicocorpus2 = json.load(mondic)
if not os.path.exists(os.path.join(dossier_corpus,"extracted_objects")):
  os.makedirs(os.path.join(dossier_corpus,"extracted_objects"))

for nomfichier in dicocorpus2 :
  compteur = 0
  nomfichier2 = str(nomfichier.split("/")[-1])
  image = cv2.imread(nomfichier)
  for objet in dicocorpus2[nomfichier].values():
    y1 = int(objet["regions"][0])
    y2 = int(objet["regions"][2])
    x1 = int(objet["regions"][1])
    x2 = int(objet["regions"][3])
    imagecropped = image[y1:y2, x1:x2] # [Y1:Y2, X1:X2] Le vecteur est sous forme [Y1 X1 Y2 X2]
    filnam = os.path.join(dossier_corpus,"extracted_objects",str(nomfichier2)+"_objet"+str(compteur)+"_"+str(objet["classe"])+".jpg")
    cv2.imwrite(filnam, imagecropped)
   # cv2_imshow(imagecropped)
    compteur += 1

print("objects extracted in ", os.path.join(dossier_corpus,"extracted_objects"))